# Multiple Linear Regression in Statsmodels - Lab

## Introduction
In this lab, you'll practice fitting a multiple linear regression model on our Boston Housing Data set!

## Objectives
You will be able to:
* Run linear regression on Boston Housing dataset with all the predictors
* Interpret the parameters of the multiple linear regression model

## The Boston Housing Data

We pre-processed the Boston Housing Data again. This time, however, we did things slightly different:
- We dropped "ZN" and "NOX" completely
- We categorized "RAD" in 3 bins and "TAX" in 4 bins
- We used min-max-scaling on "B", "CRIM" and "DIS" (and logtransformed all of them first, except "B")
- We used standardization on "AGE", "INDUS", "LSTAT" and "PTRATIO" (and logtransformed all of them first, except for "AGE") 

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
boston_features = boston_features.drop(["NOX","ZN"],axis=1)

# first, create bins for based on the values observed. 3 values will result in 2 bins
bins = [0,6,  24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# first, create bins for based on the values observed. 4 values will result in 3 bins
bins = [0, 270, 360, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix="TAX")
rad_dummy = pd.get_dummies(bins_rad, prefix="RAD")
boston_features = boston_features.drop(["RAD","TAX"], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)

In [104]:
age = boston_features["AGE"]
b = boston_features["B"]
logcrim = np.log(boston_features["CRIM"])
logdis = np.log(boston_features["DIS"])
logindus = np.log(boston_features["INDUS"])
loglstat = np.log(boston_features["LSTAT"])
logptratio = np.log(boston_features["PTRATIO"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["CRIM"] = (logcrim-min(logcrim))/(max(logcrim)-min(logcrim))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["AGE"] = (age-np.mean(age))/np.sqrt(np.var(age))
boston_features["INDUS"] = (logindus-np.mean(logindus))/np.sqrt(np.var(logindus))
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
boston_features["PTRATIO"] = (logptratio-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

In [105]:
boston_features.head()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,"RAD_(0, 6]","RAD_(6, 24]","TAX_(0, 270]","TAX_(270, 360]","TAX_(360, 712]"
0,0.000000,-1.704344,0.0,6.575,-0.120013,0.542096,-1.443977,1.000000,-1.275260,1,0,0,1,0
1,0.153211,-0.263239,0.0,6.421,0.367166,0.623954,-0.230278,1.000000,-0.263711,1,0,1,0,0
2,0.153134,-0.263239,0.0,7.185,-0.265812,0.623954,-0.230278,0.989737,-1.627858,1,0,1,0,0
3,0.171005,-1.778965,0.0,6.998,-0.809889,0.707895,0.165279,0.994276,-2.153192,1,0,1,0,0
4,0.250315,-1.778965,0.0,7.147,-0.511180,0.707895,0.165279,1.000000,-1.162114,1,0,1,0,0


## Run an linear model in Statsmodels

In [11]:
X = boston_features
y = pd.DataFrame(boston.target, columns= ["price"])

In [12]:
import statsmodels.api as sm
X_int = sm.add_constant(X)
model = sm.OLS(y,X_int).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.727
Model:                            OLS   Adj. R-squared:                  0.721
Method:                 Least Squares   F-statistic:                     109.6
Date:                Sun, 25 Nov 2018   Prob (F-statistic):          1.18e-130
Time:                        13:54:14   Log-Likelihood:                -1511.4
No. Observations:                 506   AIC:                             3049.
Df Residuals:                     493   BIC:                             3104.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             13.2243      2.449      5.399      0.000       8.412      18.036
CRIM              -0.0930      0.032     -2.893      0.004      -0.156      -0.030
INDUS             -0.1215      0.058     -2.095      0.037      -0.235      -0.008
CHAS               2.8381      0.881      3.222      0.001       1.107       4.569
RM                 3.9389      0.430      9.168      0.000       3.095       4.783
AGE               -0.0191      0.013     -1.471      0.142      -0.045       0.006
DIS               -0.8960      0.181     -4.953      0.000      -1.251      -0.541
PTRATIO           -0.8965      0.120     -7.449      0.000      -1.133      -0.660
B                  0.0104      0.003      3.804      0.000       0.005       0.016
LSTAT             -0.5381      0.052    -10.408      0.000      -0.640      -0.436
RAD_(0, 6]         5.5988      1.160      4.827      0.000       3.320       7.878
RAD_(6, 24]        7.6255      1.357      5.620      0.000       4.960      10.291
TAX_(0, 270]       5.8802      0.932      6.311      0.000       4.050       7.711
TAX_(270, 360]     3.5436      0.892      3.974      0.000       1.792       5.296
TAX_(360, 712]     3.8004      0.915      4.152      0.000       2.002       5.599
==============================================================================
Omnibus:                      189.248   Durbin-Watson:                   1.016
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              941.922
Skew:                           1.581   Prob(JB):                    2.91e-205
Kurtosis:                       8.889   Cond. No.                     4.29e+18
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.86e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

## Run the same model in Scikit-learn

In [4]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [5]:
# coefficients
linreg.coef_

array([[-0.09303484, -0.12148657,  2.83814283,  3.93893527, -0.01911545,
        -0.89596881, -0.89654594,  0.01037244, -0.53807286, -1.01333427,
         1.01333427,  1.47214372, -0.8644679 , -0.60767582]])

In [6]:
# intercept
linreg.intercept_

array([24.24452376])

## Remove the necessary variables to make sure the coefficients are the same for Scikit-learn vs Statsmodels

In [7]:
X_smaller = X.drop(["RAD_(0, 6]", "TAX_(270, 360]"], axis=1)
X_smaller.head()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,"RAD_(6, 24]","TAX_(0, 270]","TAX_(360, 712]"
0,0.00632,2.31,0.0,6.575,65.2,4.0900,15.3,396.90,4.98,0,0,0
1,0.02731,7.07,0.0,6.421,78.9,4.9671,17.8,396.90,9.14,0,1,0
2,0.02729,7.07,0.0,7.185,61.1,4.9671,17.8,392.83,4.03,0,1,0
3,0.03237,2.18,0.0,6.998,45.8,6.0622,18.7,394.63,2.94,0,1,0
4,0.06905,2.18,0.0,7.147,54.2,6.0622,18.7,396.90,5.33,0,1,0


### Statsmodels

In [8]:
import statsmodels.api as sm
X_int_sm = sm.add_constant(X_smaller)
model = sm.OLS(y,X_int_sm).fit()
# model.summary()

In [9]:
model.params

const             22.366722
CRIM              -0.093035
INDUS             -0.121487
CHAS               2.838143
RM                 3.938935
AGE               -0.019115
DIS               -0.895969
PTRATIO           -0.896546
B                  0.010372
LSTAT             -0.538073
RAD_(6, 24]        2.026669
TAX_(0, 270]       2.336612
TAX_(360, 712]     0.256792
dtype: float64

### Scikit-learn

In [10]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_smaller, y)
print(linreg.intercept_)
print(linreg.coef_)

[22.36672159]
[[-0.09303484 -0.12148657  2.83814283  3.93893527 -0.01911545 -0.89596881
  -0.89654594  0.01037244 -0.53807286  2.02666854  2.33661163  0.25679208]]


## Interpret the coefficients for PTRATIO, PTRATIO, LSTAT

- CRIM: per capita crime rate by town
- INDUS: proportion of non-retail business acres per town
- CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
- RM: average number of rooms per dwelling
- AGE: proportion of owner-occupied units built prior to 1940
- DIS: weighted distances to five Boston employment centres
- RAD: index of accessibility to radial highways
- TAX: full-value property-tax rate per $10,000
- PTRATIO: pupil-teacher ratio by town
- B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT: % lower status of the population

## Predict the house price given the following characteristics (before manipulation!!)

Make sure to transform your variables as needed!

- CRIM: 0.15
- INDUS: 6.07
- CHAS: 1        
- RM:  6.1
- AGE: 33.2
- DIS: 7.6
- PTRATIO: 17
- B: 383
- LSTAT: 10.87
- RAD: 8
- TAX: 284

## Summary
Congratulations! You've fitted your first multiple linear regression model on the Boston Housing Data.